In [23]:
!pip install simpletransformers


In [24]:
from pickle import NONE
import pandas as pd

df = pd.read_csv('/content/Saliency_train_set - final_redacted_saliency_data.csv')

df['label'] = df['saliency'].apply(lambda x: 0 if pd.isna(x) or pd.isnull(x) else 1)

df.head()



,conversation_id,utterance_id,speaker,text,saliency,label
0,742555640001,0,agent,"May I know your name, please?",NaN,0
1,742555640001,1,customer,"Uh yes, my name is {full_name:***** ****}.",NaN,0
2,742555640001,2,agent,", {first_name:*****}. How's your day going, {f...",NaN,0
3,742555640001,3,customer,It's going good. Thank you for asking.,NaN,0
4,742555640001,4,agent,That's great. So how can I help you?,NaN,0


In [25]:
from sklearn.model_selection import train_test_split

X=df['text'].values
Y=df['label'].values
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [26]:
train_data=[]
for i in range(len(X_train)):
  l=[X_train[i],Y_train[i]]
  train_data.append(l)

train_df = pd.DataFrame(train_data)
train_df.columns = ["text", "label"]

In [27]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.metrics import f1_score
import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model_args = ClassificationArgs()
model_args.num_train_epochs=5
model_args.overwrite_output_dir=True
model_args.adafactor_eps=(1e-4,1e-3)
model_args.learning_rate=7e-5

model = ClassificationModel("bert", "bert-base-cased", args=model_args,use_cuda=True)
model.train_model(train_df,acc=f1_score)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/623 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/78 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/78 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/78 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/78 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/78 [00:00<?, ?it/s]

(390, 0.27664693761378145)

In [28]:
result, model_outputs, wrong_predictions = model.eval_model(train_df,acc=f1_score)
print(result)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/623 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/78 [00:00<?, ?it/s]

{'mcc': 0.9546972647657829, 'tp': 333, 'tn': 276, 'fp': 6, 'fn': 8, 'auroc': 0.9972234354526736, 'auprc': 0.9973467709065589, 'acc': 0.9794117647058823, 'eval_loss': 0.08304498680174732}


In [29]:
pred,_ = model.predict(["Hey,my phone is facing technical issues"])
print(pred)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[1]


In [30]:
pred = model.predict(["Good afternoon! How can I help you?","Hi, I'd like to inquire about your pricing plans.","Of course! What specific information are you looking for?","I want to know if you offer any discounts for annual subscriptions."])
print(pred)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

(array([0, 1, 0, 1]), array([[ 3.52929688, -3.35546875],
       [-2.37890625,  2.20117188],
       [ 2.91796875, -3.1875    ],
       [-2.14257812,  2.00976562]]))


In [31]:
eval_data=[]
for i in range(len(X_test)):
  l=[X_test[i],Y_test[i]]
  eval_data.append(l)
print(len(eval_data))
eval_df = pd.DataFrame(eval_data)
eval_df.columns = ["text", "labels"]

result, model_outputs, wrong_predictions = model.eval_model(eval_df,acc=f1_score)
print(result)

156


  0%|          | 0/156 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/20 [00:00<?, ?it/s]

{'mcc': 0.8318546464587009, 'tp': 79, 'tn': 64, 'fp': 6, 'fn': 7, 'auroc': 0.9443521594684385, 'auprc': 0.9562785675841248, 'acc': 0.9239766081871345, 'eval_loss': 0.42436645925045013}


In [32]:
# model.save_model("/content/heybuddy/")
model.model.save_pretrained('model1')
model.tokenizer.save_pretrained('model1')
model.config.save_pretrained('model1/')

In [37]:
model1 = ClassificationModel("bert","model1")

In [36]:
pred = model1.predict(["Good afternoon! How can I help you?","Hi, I'd like to inquire about your pricing plans.","Of course! What specific information are you looking for?","I want to know if you offer any discounts for annual subscriptions."])
print(pred)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

(array([0, 1, 0, 1]), array([[ 3.52929688, -3.35546875],
       [-2.37890625,  2.20117188],
       [ 2.91796875, -3.1875    ],
       [-2.14257812,  2.00976562]]))
